# Adaptive PDE discretizations on cartesian grids 
## Volume : Reproducible research
## Part : Eikonal CPU/GPU solvers comparison
## Chapter : Manifolds defined by local charts

We solve eikonal PDEs on manifolds defined by local charts. In particular the projective plane, and the klein bottle.

This functionality is so far only implemented in the GPU eikonal solver. It is used in the context of the rolling ball models, which involve PDEs on the group $\mathrm{SO}_3$, parametrized by three dimensional projective space $\mathrm{P}_3$.

## 0. Importing the required libraries

In [2]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Trailers','GPU'))

In [3]:
from agd import Eikonal
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import LinearParallel as lp
norm_infinity = ad.Optimization.norm_infinity

In [5]:
from agd.ExportedCode.Notebooks_Algo import RollingBall_Models
plane_to_sphere = RollingBall_Models.quaternion_from_euclidean
sphere_to_plane = RollingBall_Models.euclidean_from_quaternion

In [6]:
import numpy as np; xp=np; allclose = np.allclose
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

### 0.1 Additional configuration

The GPU accelerated solver is mandatory for this notebook.

In [4]:
xp,Eikonal,plt,allclose = [ad.cupy_friendly(e) for e in (xp,Eikonal,plt,allclose)]

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Setting dictIn.default_mode = 'gpu' in module agd.Eikonal .
Returning a copy of module matplotlib.pyplot whose functions accept cupy arrays as input.
Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


## 1. Projective plane



### 1.1 Manifold description

The projective plane is defined as the unit sphere, in which pairs of opposite points are identified.
The `plane_to_sphere` map parametrizes the northern hemisphere from the equatorial disk, by projection from the south pole.

In [8]:
def chart_mapping(x): 
    """
    Glue map for the projective plane, as parameterized from the equator.
    (Maps a point to the other point with which it identifies.)
    """    
    return sphere_to_plane( - plane_to_sphere(x))

The equatorial projection is conformal to the isotropic metric on the plane, with an explicit cost function.

In [9]:
def cost(x):
    """
    Isotropic cost function for the projective plane, as parametrized from the equator.
    """
    return 1/np.sqrt(1.+lp.dot_VV(x,x))

### 1.2 Input to the library

In [10]:
hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5,0.7],
    'exportValues':True,
    'factoringRadius':10,
})

r=1.1
hfmIn.SetRect([[-r,r],[-r,r]],dimx=101)
X = hfmIn.Grid()

hfmIn['cost']=cost(X)
hfmIn.SetUniformTips((4,4))

